In [10]:
#firstly, extracting all items inside a user's iventory (only cs2 items)
import requests
import time
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/117.0.0.0 Safari/537.36"
}
def fetch_inventory_page(steamid64: str, appid: int, contextid: int, start: str = None):
    url = f"https://steamcommunity.com/inventory/{steamid64}/{appid}/{contextid}"
    resp = requests.get(url,headers=HEADERS, timeout=15)
    resp.raise_for_status()
    return resp.json()
def inventory_items_for_user(steamid64: str, appid: int, contextid: int, delay_between_requests: float = 0.5):
    items = []
    seen_assetids = set()
    start = None

    while True:
        data = fetch_inventory_page(steamid64, appid, contextid, start)
        # check if inventory is private or empty
        if not data or "assets" not in data or "descriptions" not in data:
            # Might be private or zero items; return whatever we have or raise
            return items, data

        assets = data.get("assets", [])
        descriptions = data.get("descriptions", [])

        # Build lookup for descriptions by (classid, instanceid)
        desc_lookup = {}
        for d in descriptions:
            key = (str(d.get("classid")), str(d.get("instanceid", "0")))
            desc_lookup[key] = d

        for a in assets:
            assetid = a.get("assetid")  # unique per item
            if assetid in seen_assetids:
                continue
            seen_assetids.add(assetid)

            classid = str(a.get("classid"))
            instanceid = str(a.get("instanceid", "0"))
            desc = desc_lookup.get((classid, instanceid)) or {}

            item = {
                "assetid": assetid,
                "classid": classid,
                "instanceid": instanceid,
                "amount": a.get("amount", 1),
                "market_tradable_restriction": a.get("market_tradable_restriction"),
                # descriptive fields:
                "name": desc.get("name"),
                "type": desc.get("type"),
                "market_hash_name": desc.get("market_hash_name"),
                "tradable": desc.get("tradable"),
                "tags": desc.get("tags", []),
                "marketable": desc.get("marketable"),
                # entire desc for debugging
                "description_raw": desc
            }
            if item["tradable"] ==1 and item["marketable"] == 1:
                items.append(item)

        # pagination
        more = data.get("more_items") or False
        # some responses give 'last_assetid' to use as start; others use 'more_start'
        if more:
            start = data.get("last_assetid") or data.get("more_start")
            if not start:
                # fallback: if server indicates 'more_items' but no token, stop to avoid infinite loop
                break
            time.sleep(delay_between_requests)  # be polite
        else:
            break

    return items, data

In [11]:
steamid64 = "76561198923974658"   # example SteamID64
appid = 730                       # CS:GO/CS2 (confirm)
contextid = 2                     

items, raw = inventory_items_for_user(steamid64, appid, contextid)
print(f"Found {len(items)} items")
for i, it in enumerate(items[:200], start=1):
    print(f"{i}. {it['name']!r} {it['market_hash_name']!r} ({it['assetid']}) - tradable={it.get('tradable')} marketable={it.get('marketable')}")

Found 50 items
1. 'G3SG1 | VariCamo' 'G3SG1 | VariCamo (Field-Tested)' (47821593041) - tradable=1 marketable=1
2. 'Recoil Case' 'Recoil Case' (47821593040) - tradable=1 marketable=1
3. 'SG 553 | Night Camo' 'SG 553 | Night Camo (Battle-Scarred)' (47655223284) - tradable=1 marketable=1
4. 'Revolution Case' 'Revolution Case' (47655223283) - tradable=1 marketable=1
5. 'SSG 08 | Blue Spruce' 'SSG 08 | Blue Spruce (Field-Tested)' (47501101767) - tradable=1 marketable=1
6. 'Dreams & Nightmares Case' 'Dreams & Nightmares Case' (47501101766) - tradable=1 marketable=1
7. 'StatTrak™ MAG-7 | Insomnia' 'StatTrak™ MAG-7 | Insomnia (Battle-Scarred)' (47394323285) - tradable=1 marketable=1
8. 'Sticker | Aleksib (Gold) | Copenhagen 2024' 'Sticker | Aleksib (Gold) | Copenhagen 2024' (47386508166) - tradable=1 marketable=1
9. 'StatTrak™ M4A4 | Choppa' 'StatTrak™ M4A4 | Choppa (Field-Tested)' (47381099310) - tradable=1 marketable=1
10. 'FAMAS | Meow 36' 'FAMAS | Meow 36 (Minimal Wear)' (47380823343) - tr

In [36]:
item_summaries = []#this for latter fetch on marketoverpice
seen_names = set()#this for checking duplicates market_hash_name
for i, it in enumerate(items[:200], start=1):
    name = it['market_hash_name']
    #skip if name are already added
    if not name or name in seen_names:
        continue
    seen_names.add(name)
    summary = {
        "index": i,
        "name": it['name'],
        "market_hash_name": it['market_hash_name'],
        "tradable": it.get('tradable'),
        "marketable": it.get('marketable')
    }
    item_summaries.append(summary)

print(item_summaries)

[{'index': 1, 'name': 'G3SG1 | VariCamo', 'market_hash_name': 'G3SG1 | VariCamo (Field-Tested)', 'tradable': 1, 'marketable': 1}, {'index': 2, 'name': 'Recoil Case', 'market_hash_name': 'Recoil Case', 'tradable': 1, 'marketable': 1}, {'index': 3, 'name': 'SG 553 | Night Camo', 'market_hash_name': 'SG 553 | Night Camo (Battle-Scarred)', 'tradable': 1, 'marketable': 1}, {'index': 4, 'name': 'Revolution Case', 'market_hash_name': 'Revolution Case', 'tradable': 1, 'marketable': 1}, {'index': 5, 'name': 'SSG 08 | Blue Spruce', 'market_hash_name': 'SSG 08 | Blue Spruce (Field-Tested)', 'tradable': 1, 'marketable': 1}, {'index': 6, 'name': 'Dreams & Nightmares Case', 'market_hash_name': 'Dreams & Nightmares Case', 'tradable': 1, 'marketable': 1}, {'index': 7, 'name': 'StatTrak™ MAG-7 | Insomnia', 'market_hash_name': 'StatTrak™ MAG-7 | Insomnia (Battle-Scarred)', 'tradable': 1, 'marketable': 1}, {'index': 8, 'name': 'Sticker | Aleksib (Gold) | Copenhagen 2024', 'market_hash_name': 'Sticker | A

In [ ]:
from playwright.async_api import async_playwright
async def login_site_and_get_cookies():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)  # headless=False opens a visible window
        context = await browser.new_context()

        page = await browser.new_page()
        await page.goto("http://buff.163.com")

        # Wait for login button or perform manual login
        await page.click("text=Login/Register")
        # Prepare to catch the popup (Steam login)
        async with page.expect_popup() as popup_info:
            await page.click("text=Other login methods")
            await page.wait_for_timeout(10000)
        #inputing account's info ( im using an freshly created account, without steam guard and such)
        #steam's account need to be linked on a registered buff's account.
        popup = await popup_info.value
        await popup.fill("input[type='text']", "account_")
        await popup.fill("input[type='password']", "password")
        await popup.wait_for_timeout(10000)
        await popup.wait_for_selector("button:has-text('Sign In')")
        await popup.click("button:has-text('Sign In')")
        await popup.wait_for_timeout(10000)
        await popup.locator('input[value="Sign In"]').click()

        cookies = await page.context.cookies()
        target_domain = "buff.163.com"
        filtered_cookies = []
        print(cookies)
        for cookie in cookies:
            if cookie["domain"] == target_domain:
                filtered_cookies.append(cookie)
        await page.wait_for_timeout(20000)  # wait 20 seconds for you to log in

        # Close the browser when done
        await browser.close()
        return filtered_cookies

In [51]:
#now, use the name of items, and fetch the current market price
#Note: we do need cookies for this request.
from urllib.parse import quote
HEADERS2 = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/120.0.0.0 Safari/537.36",
    "Referer": "https://buff.163.com/",
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "X-Requested-With": "XMLHttpRequest"
}
def get_market_price(market_hash_name,cookies_list):
    print(market_hash_name)
    base_url = "https://buff.163.com/api/market/goods"
    url = base_url + f"?game=csgo&page_num=1&search=" +quote(market_hash_name)
    #We need the cookies input manually, or getting the cookies on a logged on browser onto steam automatically
    #Seems like buff also using session. We can copy and extract the cookies from an logged in browser, but for automation process, its not practical.
    # cookie_str = f""
    cookies = {c['name']: c['value'] for c in cookies_list}
    resp = requests.get(url,headers=HEADERS,cookies= cookies)
    #update: we wont need cookies, just change the url to http, instead of https.
    # resp = requests.get(url,headers=HEADERS2)
    if (resp.status_code == 200):
        data = resp.json()
    
    if "data" not in data or not data["data"]["items"]:
        print("No matching items found.")
        return None
    items = data["data"]["items"]

    matched_item = None
    for item in items:
        if item["market_hash_name"] == market_hash_name:
            matched_item = item
            break
    
    if not matched_item:
        print("Item found but names do not exactly match.")
        return None
    else:
    #converting the return data to match with steam's response
        result = {
            market_hash_name: {
                "lowest_price": f"${matched_item['sell_min_price']}",
                "median_price": f"${matched_item['quick_price']}",
                "volume": str(matched_item["sell_num"])
            }
    }
        return result

    

In [ ]:
cookies_list = await login_site_and_get_cookies()

In [ ]:
import time
import random
from collections import defaultdict
data_by_timestamp = defaultdict(dict)
timestamp = int(time.time())
for item in item_summaries:
    name = item["market_hash_name"]
    price_info = get_market_price(name,cookies_list)
    
    if price_info:
        print(price_info)
        #data_by_timestamp[timestamp][name] = price_info
    #     print(f"{name}: {price_info['lowest_price']} (median: {price_info['median_price']})")
    else:
        print(f"{name}: Price not found")
    #adding delays for bypass steam's rate limiting
    time.sleep(random.uniform(4.5, 6.0))


TypeError: get_market_price() missing 1 required positional argument: 'cookies_list'

In [28]:
cookies_list = await login_site_and_get_cookies()
print(cookies_list)
# cookies = "Device-Id=OsUryWgyoylmSL2KP4ig; Locale-Supported=en; game=csgo; session=1-nSeow00GiYG2BitmdCGbVvI77JaQyTV1YlDKB42gHRgf2038416215; client_id=CTyt1OV8TI2ulw7qngZKsA; account__pre_login__steam_key=667aa491fe3fcb990058445083951a92000db2e4813d05fe4bd2be5c0d937512; csrf_token=IjA1NGNiM2I2NmZjOTM4NjQ2MmU3YTNlOGRhMmViYTY5OGQ2OGVhZWQi.aRoGHQ.gLhi5gzdYD3tDP--RCVzFZGSaxw"
# cookies = {c['name']: c['value'] for c in cookies_str}
get_market_price("Kukri Knife | Night Stripe (Field-Tested)",cookies_list)

TargetClosedError: Page.goto: Target page, context or browser has been closed
Call log:
  - navigating to "http://buff.163.com/", waiting until "load"


In [52]:
get_market_price("★ Kukri Knife | Night Stripe (Field-Tested)",cookies_list)

★ Kukri Knife | Night Stripe (Field-Tested)


{'★ Kukri Knife | Night Stripe (Field-Tested)': {'lowest_price': '$439',
  'median_price': '$438.5',
  'volume': '154'}}